In [20]:
from praw import Reddit
from praw.models import Subreddit, Submission
import os
from dotenv import load_dotenv
from datetime import datetime, date, timedelta
from dotenv import dotenv_values
import datetime as dt
import pandas as pd
from datetime import datetime, date, timedelta

In [29]:
import praw
from datetime import datetime
from dotenv import dotenv_values
import pandas as pd

# Load environment variables
config = dotenv_values(".env")

# Initialize Reddit API
reddit = praw.Reddit(
    client_id=config["CLIENT_ID"],
    client_secret=config["CLIENT_SECRET"],
    redirect_uri=config["REDIRECT_URL"],
    user_agent=config["USER_AGENT"]
)

def fetch_posts_about_keyword(subreddit_name, keywords, limit=500000):
    subreddit = reddit.subreddit(subreddit_name)
    data = []
    
    # Define the cutoff date (before November 2024)
    cutoff_date = datetime(2024, 11, 30)  # November 1, 2024
    
    # Fetch posts from subreddit
    for post in subreddit.search(' '.join(keywords), time_filter='all', limit=limit):
        post_date = datetime.utcfromtimestamp(post.created)
        
        # Check if the post is before the cutoff date
        if post_date < cutoff_date:
            matching_keywords = [keyword for keyword in keywords if keyword.lower() in post.title.lower()]

            for keyword in matching_keywords:
                post.comments.replace_more(limit=0)
                comments = post.comments.list()

                for comment in comments:
                    data.append({
                        'post_title': post.title,
                        'post_score': post.score,
                        'post_id': post.id,
                        'post_num_comments': post.num_comments,
                        'post_created': post_date,
                        'post_body': post.selftext,
                        'comment_id': comment.id,
                        'comment_body': comment.body,
                        'comment_score': comment.score,
                        'matched_keywords': ", ".join(matching_keywords)
                    })
                break  # Only process the post once per matching keyword

    return data

subreddit_name = "stocks"
keywords = ["Uber", "youtube", "meta", "apple", "nvidia", "microsoft", "amazon", "Saudi Arabian Oil Co", "intel", "tesla", "MARA Holdings"]

# Fetch posts before November 2024
data = fetch_posts_about_keyword(subreddit_name, keywords, limit=1000)

# Create DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv("posts_before_november_2024.csv", index=False)


In [30]:
df

,post_title,post_score,post_id,post_num_comments,post_created,post_body,comment_id,comment_body,comment_score,matched_keywords
0,Nvidia passes Microsoft in market cap to becom...,2035,1divtu5,512,2024-06-18 17:07:58,"Nvidia, long known in the niche gaming communi...",l96k3bm,The first trillion is always the hardest. Now ...,980,"nvidia, microsoft"
1,Nvidia passes Microsoft in market cap to becom...,2035,1divtu5,512,2024-06-18 17:07:58,"Nvidia, long known in the niche gaming communi...",l96lcmg,I keep waiting to see NVDA slide back a bit or...,523,"nvidia, microsoft"
2,Nvidia passes Microsoft in market cap to becom...,2035,1divtu5,512,2024-06-18 17:07:58,"Nvidia, long known in the niche gaming communi...",l96ukx9,Really should have invested in Nvidia when I d...,244,"nvidia, microsoft"
3,Nvidia passes Microsoft in market cap to becom...,2035,1divtu5,512,2024-06-18 17:07:58,"Nvidia, long known in the niche gaming communi...",l96oygx,"> “When I see a bubble forming, I rush in to b...",147,"nvidia, microsoft"
4,Nvidia passes Microsoft in market cap to becom...,2035,1divtu5,512,2024-06-18 17:07:58,"Nvidia, long known in the niche gaming communi...",l97ui72,I was deciding to buy between AMD and NVDA a c...,55,"nvidia, microsoft"
...,...,...,...,...,...,...,...,...,...,...
2233,Stocks are advancing ahead of earnings results...,24,9r9niy,9,2018-10-25 12:03:39,\n\n### US Stocks\n\n* **Global stocks are ad...,e8fiiir,Seriously. I’ve been buying all of the last tw...,0,"amazon, intel"
2234,Stocks are advancing ahead of earnings results...,24,9r9niy,9,2018-10-25 12:03:39,\n\n### US Stocks\n\n* **Global stocks are ad...,e8fk3jn,The children aren't used to pullbacks.They sca...,3,"amazon, intel"
2235,Stocks are advancing ahead of earnings results...,24,9r9niy,9,2018-10-25 12:03:39,\n\n### US Stocks\n\n* **Global stocks are ad...,e8frw0t,Depends what kind of stocks we are talking abo...,3,"amazon, intel"
2236,Stocks are advancing ahead of earnings results...,24,9r9niy,9,2018-10-25 12:03:39,\n\n### US Stocks\n\n* **Global stocks are ad...,e8fs10p,"I waited for solid companies like ADBE, AMZN, ...",3,"amazon, intel"


In [42]:
# Load environment variables
config = dotenv_values(".env")

# Initialize Reddit API
reddit = praw.Reddit(
    client_id=config["CLIENT_ID"],
    client_secret=config["CLIENT_SECRET"],
    user_agent=config["USER_AGENT"]
)

def fetch_posts_about_keywords(subreddit_name, keywords, limit=500000):
    subreddit = reddit.subreddit(subreddit_name)
    data = []
    
    # Define the date range for March and April (2024)
    today = datetime(2024, 9, 30, 23, 59, 59)  # End of April
    start_date = datetime(2024, 7, 1)  # Start of March
    
    # Iterate through each keyword
    for keyword in keywords:
        print(f"Fetching posts for keyword: {keyword}")
        for post in subreddit.search(keyword, time_filter='all', limit=limit):
            post_date = datetime.utcfromtimestamp(post.created)
            
            # Check if the post is within the desired time range
            if start_date <= post_date <= today:
                post.comments.replace_more(limit=0)
                comments = post.comments.list()

                for comment in comments:
                    data.append({
                        'post_title': post.title,
                        'post_score': post.score,
                        'post_id': post.id,
                        'post_num_comments': post.num_comments,
                        'post_created': post_date,
                        'post_body': post.selftext,
                        'comment_id': comment.id,
                        'comment_body': comment.body,
                        'comment_score': comment.score,
                        'matched_keyword': keyword
                    })
    
    return data

subreddit_name = "stocks"
keywords = ["Uber", "youtube", "meta", "apple", "nvidia", "microsoft", "amazon", "Saudi Arabian Oil Co", "intel", "tesla", "MARA Holdings"]

# Fetch posts from March and April 2024
data = fetch_posts_about_keywords(subreddit_name, keywords, limit=500000)

# Create DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv("before_september_reddit_comments.csv", index=False)


Fetching posts for keyword: Uber
Fetching posts for keyword: youtube
Fetching posts for keyword: meta
Fetching posts for keyword: apple
Fetching posts for keyword: nvidia
Fetching posts for keyword: microsoft
Fetching posts for keyword: amazon
Fetching posts for keyword: Saudi Arabian Oil Co
Fetching posts for keyword: intel
Fetching posts for keyword: tesla
Fetching posts for keyword: MARA Holdings


In [43]:
df

,post_title,post_score,post_id,post_num_comments,post_created,post_body,comment_id,comment_body,comment_score,matched_keyword
0,Uber and Waymo to offer driverless ride-sharin...,218,1ffw4rb,83,2024-09-13 14:29:17,Uber announced Friday it is expanding its part...,lmxzxby,This is really interesting to me because isn't...,30,Uber
1,Uber and Waymo to offer driverless ride-sharin...,218,1ffw4rb,83,2024-09-13 14:29:17,Uber announced Friday it is expanding its part...,lmyho7c,Clearly Waymo wants to license its technology ...,37,Uber
2,Uber and Waymo to offer driverless ride-sharin...,218,1ffw4rb,83,2024-09-13 14:29:17,Uber announced Friday it is expanding its part...,lmz6qiq,Despite the narrative that driverless tech is ...,9,Uber
3,Uber and Waymo to offer driverless ride-sharin...,218,1ffw4rb,83,2024-09-13 14:29:17,Uber announced Friday it is expanding its part...,lmxw2fk,Sort of related - month ago I mentioned (https...,16,Uber
4,Uber and Waymo to offer driverless ride-sharin...,218,1ffw4rb,83,2024-09-13 14:29:17,Uber announced Friday it is expanding its part...,lmzo0cc,"If successful, I hope either company decides t...",4,Uber
...,...,...,...,...,...,...,...,...,...,...
33449,These are the stocks on my watchlist (8/1),35,1ehgi7k,17,2024-08-01 13:20:18,Hi! I am an ex-prop shop equity trader. This i...,lfzblj8,I have more shares of NVDL because I sell cove...,4,MARA Holdings
33450,These are the stocks on my watchlist (8/1),35,1ehgi7k,17,2024-08-01 13:20:18,Hi! I am an ex-prop shop equity trader. This i...,lg01jpi,"It's doing well, but as expected I have a few ...",2,MARA Holdings
33451,These are the stocks on my watchlist (8/1),35,1ehgi7k,17,2024-08-01 13:20:18,Hi! I am an ex-prop shop equity trader. This i...,lfzce7c,"Fair enough, if you're cognizant of the risks ...",4,MARA Holdings
33452,These are the stocks on my watchlist (8/1),35,1ehgi7k,17,2024-08-01 13:20:18,Hi! I am an ex-prop shop equity trader. This i...,lg38j33,>I lost about 20K a few years back for falling...,2,MARA Holdings
